In [ ]:
import tensorflow as tf
COLOR_MODE = "grayscale"
CHANNELS = 1 if COLOR_MODE == "grayscale" else 3
IMAGE_HEIGHT = 100
IMAGE_WIDTH = 100
BATCH_SIZE = 256
INTERPOLATION = "bilinear"
TRAINING_PATH = "./dataset/images"
TESTING_PATH = "./dataset/Test_Set/Test_Set"
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_PATH,
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    subset='training',
    validation_split=0.2,
    seed=42,
    interpolation=INTERPOLATION,
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_PATH,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    subset='validation',
    validation_split=0.2,
    seed=42,
    interpolation=INTERPOLATION,
)
print(train_ds)

In [ ]:
class_names = train_ds.class_names
print(class_names)
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    print(images[i].shape)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
import keras
model = keras.models.Sequential([
    keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(256, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(4, activation="softmax"),
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
model.summary(
    expand_nested=True,
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Compute class weights
y_train = np.concatenate([y for x, y in train_ds], axis=0) 
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# Convert class weights to a dictionary format
class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

In [ ]:
import tensorflow as tf
EPOCHS = 100
PATIENCE = 25
earlystop_loss = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=test_ds,
    callbacks=[earlystop_loss],
    class_weight=class_weights_dict,
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)

In [ ]:
from matplotlib import pyplot as plt
print("Epochs run:", len(history.history["loss"]))
acc = history.history["sparse_categorical_accuracy"]
val_acc = history.history["val_sparse_categorical_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
# Train and validation accuracy
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.ylim((0, 1))
plt.plot(epochs, acc, label="Training accurarcy")
plt.plot(epochs, val_acc, label="Validation accurarcy")
plt.title("Training and Validation accurarcy")
plt.legend()
# Train and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label="Training loss")
plt.plot(epochs, val_loss, label="Validation loss")
plt.title("Training and Validation loss")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
y_pred = []
y_true = []
# iterate over the dataset
for image_batch, label_batch in test_ds:   # use dataset.unbatch() with repeat
   y_true.append(label_batch)
   preds = model.predict(image_batch)
   y_pred.append(np.argmax(preds, axis = -1))
# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)
print("predicted_labels", len(predicted_labels), predicted_labels)
print("true_labels", len(correct_labels), correct_labels)
# Build the confusion matrix
conf_matrix = confusion_matrix(correct_labels, predicted_labels)
print(conf_matrix)
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(4), yticklabels=range(4))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(f"Classification Report :- \n {classification_report(correct_labels, predicted_labels)}")

In [ ]:
# save the iris classification model
model.save("covid-cnn.h5")

In [ ]:
import tensorflow as tf
model.save("covid-cnn.h5") 
loaded_model = tf.keras.models.load_model("covid-cnn.h5") 
loss, accuracy = loaded_model.evaluate(test_ds) 

In [ ]:
import numpy as np
from sklearn import metrics
y = np.array(correct_labels)
scores = np.array(predicted_labels)
# Check unique labels and count positive samples
unique_labels = np.unique(y)
positive_count = np.sum(y == 9)
print("Unique labels in y:", unique_labels)
print("Number of positive samples (label=9):", positive_count)
# fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=9)
# roc_auc = metrics.auc(fpr, tpr)

if positive_count == 0:
    print("No positive samples in y_true. Cannot calculate ROC curve.")
else:
    fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=None)
    roc_auc = metrics.auc(fpr, tpr)

    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()